In [ ]:
#!sudo apt update && sudo apt install ffmpeg
#!sudo apt-get install libportaudio2 portaudio19-dev
#!sudo apt install python3-gi python3-gi-cairo gir1.2-gobject-2.0 gir1.2-gstreamer-1.0 gstreamer1.0-plugins-good gstreamer1.0-plugins-base gstreamer1.0-gl

In [ ]:
!pip install openai-whisper sounddevice scipy ollama pyttsx3 numpy

In [ ]:
!pip install edge-tts playsound==1.2.2

In [ ]:
!pip install pydub

In [20]:
!pip install noisereduce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 2.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 5.2 MB/s eta 0:00:00a 0:00:01


In [9]:
#!sudo apt install build-essential python3-dev
!pip install webrtcvad
!pip install webrtcvad-wheels
!pip install --upgrade setuptools

In [4]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [1]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import whisper
import ollama
import edge_tts
from pydub import AudioSegment
import os
import sys
import time
import asyncio
import webrtcvad
from collections import deque
import io
import wave
import string
import noisereduce as nr

/home/rafadom/2ºCuatrimestre/TFM-main/test reconocimiento de audio/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:

# --- Configuración General ---
WHISPER_MODEL = "small"
OLLAMA_MODEL = "gemma3" # O el modelo específico que usas
TEMP_AUDIO_FILE_TTS = "temp_tts_output_notebook.mp3"
EXIT_KEYWORDS = ["adiós", "terminar", "salir", "hasta luego", "bye", "exit", "adios"]

# --- Configuración de Audio y VAD ---
VAD_SAMPLE_RATE = 16000
VAD_FRAME_MS = 30
VAD_CHUNK_SIZE = (VAD_SAMPLE_RATE * VAD_FRAME_MS) // 1000
VAD_AGGRESSIVENESS = 1
SILENCE_DURATION_S = 0.8
MIN_SPEECH_DURATION_S = 0.3
PADDING_S = 0.2
MICROPHONE_DEVICE_INDEX = None

# --- Configuración de Voz TTS (edge-tts) ---
TTS_VOICE = "es-ES-ElviraNeural"

# --- System Prompt (Define la personalidad/instrucciones base) ---
# Se enviará con CADA pregunta al LLM, pero sin historial previo.
try:
    current_time = time.time()
    current_date_es = time.strftime('%A, %d de %B de %Y', time.localtime(current_time))
    current_location = "Almería, Andalucía, España"
except Exception:
    current_date_es = "una fecha actual"
    current_location = "tu ubicación actual"
SYSTEM_PROMPT = f"Eres un asistente de IA llamado Jarvis. Responde de forma concisa y útil en español de España a la pregunta del usuario. Ignora cualquier conversación anterior. Hoy es {current_date_es}."

# --- Variables Globales y Evento Async ---
audio_buffer = deque()
recording_complete_event = asyncio.Event()
is_speaking_global = False
speech_frames_count_global = 0
silence_frames_count_global = 0

In [10]:
# --- Inicialización VAD ---
try:
    vad = webrtcvad.Vad(VAD_AGGRESSIVENESS)
except Exception as e:
    print(f"Error inicializando VAD: {e}")
    raise

# --- Inicialización Whisper ---
print("Cargando modelo Whisper...")
try:
    whisper_model = whisper.load_model(WHISPER_MODEL)
    print(f"Modelo Whisper '{WHISPER_MODEL}' cargado.")
except Exception as e:
    print(f"Error cargando el modelo Whisper: {e}")
    raise

Cargando modelo Whisper...
Modelo Whisper 'small' cargado.


In [11]:
# --- Listar Dispositivos de Audio (para diagnóstico) ---
print("Buscando dispositivos de audio...")
try:
    print(sd.query_devices())
    default_input_device = sd.default.device[0]
    print(f"Dispositivo de entrada predeterminado del sistema: {default_input_device}")
    if MICROPHONE_DEVICE_INDEX is None:
        MICROPHONE_DEVICE_INDEX = default_input_device
    print(f"Usando dispositivo de entrada con índice: {MICROPHONE_DEVICE_INDEX}")
except Exception as e:
    print(f"Error al consultar dispositivos de audio: {e}. Usando predeterminado si es posible.")
    # No salimos, intentaremos continuar con el predeterminado si falla la consulta

Buscando dispositivos de audio...
   0 HDA NVidia: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   1 HDA NVidia: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
   2 HDA NVidia: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   3 HDA NVidia: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   4 sof-hda-dsp: - (hw:1,0), ALSA (2 in, 2 out)
   5 sof-hda-dsp: - (hw:1,3), ALSA (0 in, 2 out)
   6 sof-hda-dsp: - (hw:1,4), ALSA (0 in, 2 out)
   7 sof-hda-dsp: - (hw:1,5), ALSA (0 in, 2 out)
   8 sof-hda-dsp: - (hw:1,6), ALSA (2 in, 0 out)
   9 sof-hda-dsp: - (hw:1,7), ALSA (2 in, 0 out)
  10 hdmi, ALSA (0 in, 8 out)
  11 pulse, ALSA (32 in, 32 out)
* 12 default, ALSA (32 in, 32 out)
Dispositivo de entrada predeterminado del sistema: 12
Usando dispositivo de entrada con índice: 12


In [12]:
# --- Funciones ---

def _audio_callback(indata, frames, time_info, status):
    """Callback de sounddevice, procesa audio en tiempo real con VAD."""
    global audio_buffer, is_speaking_global, speech_frames_count_global, silence_frames_count_global, recording_complete_event
    if status: print(f"Status del stream de audio: {status}", file=sys.stderr)
    clipped_indata = np.clip(indata, -1.0, 1.0)
    indata_int16 = (clipped_indata * 32767).astype(np.int16)
    audio_chunk_bytes = indata_int16.tobytes()
    bytes_per_sample = 2
    samples_per_vad_chunk = VAD_CHUNK_SIZE
    bytes_per_vad_chunk = samples_per_vad_chunk * bytes_per_sample
    for i in range(0, len(audio_chunk_bytes), bytes_per_vad_chunk):
        chunk_vad = audio_chunk_bytes[i : i + bytes_per_vad_chunk]
        if len(chunk_vad) < bytes_per_vad_chunk: continue
        try: is_speech = vad.is_speech(chunk_vad, VAD_SAMPLE_RATE)
        except Exception: is_speech = False
        if is_speech:
            silence_frames_count_global = 0
            if not is_speaking_global:
                is_speaking_global = True; speech_frames_count_global = 1
                padding_frames = int((PADDING_S * VAD_SAMPLE_RATE) / VAD_CHUNK_SIZE)
                for _ in range(padding_frames): audio_buffer.append(b'\x00' * bytes_per_vad_chunk)
            audio_buffer.append(chunk_vad); speech_frames_count_global += 1
        else:
            if is_speaking_global:
                silence_frames_count_global += 1; audio_buffer.append(chunk_vad)
                silence_threshold_frames = int((SILENCE_DURATION_S * VAD_SAMPLE_RATE) / VAD_CHUNK_SIZE)
                if silence_frames_count_global >= silence_threshold_frames:
                    is_speaking_global = False
                    padding_frames = int((PADDING_S * VAD_SAMPLE_RATE) / VAD_CHUNK_SIZE)
                    for _ in range(padding_frames): audio_buffer.append(b'\x00' * bytes_per_vad_chunk)
                    min_speech_frames = int((MIN_SPEECH_DURATION_S * VAD_SAMPLE_RATE) / VAD_CHUNK_SIZE)
                    if speech_frames_count_global > min_speech_frames:
                        try: loop = asyncio.get_event_loop(); loop.call_soon_threadsafe(recording_complete_event.set)
                        except RuntimeError: recording_complete_event.set()
                    else: audio_buffer.clear(); speech_frames_count_global = 0
                    silence_frames_count_global = 0

async def listen_for_speech():
    """Inicia el stream de audio y espera a que VAD detecte fin de habla."""
    global audio_buffer, recording_complete_event, is_speaking_global, speech_frames_count_global, silence_frames_count_global
    audio_buffer.clear(); recording_complete_event.clear()
    is_speaking_global = False; speech_frames_count_global = 0; silence_frames_count_global = 0
    print("🎤 Escuchando...", flush=True)
    if VAD_SAMPLE_RATE not in [8000, 16000, 32000, 48000]:
         print(f"Error: Sample rate {VAD_SAMPLE_RATE}Hz no soportado por VAD.", file=sys.stderr); return None
    loop = asyncio.get_event_loop()
    try:
        stream = sd.InputStream(
            samplerate=VAD_SAMPLE_RATE, channels=1, dtype='float32', blocksize=VAD_CHUNK_SIZE,
            device=MICROPHONE_DEVICE_INDEX,
            callback=lambda indata, frames, time, status: loop.call_soon_threadsafe(_audio_callback, indata, frames, time, status))
        with stream: await recording_complete_event.wait()
        return b"".join(list(audio_buffer))
    except sd.PortAudioError as pae:
         print(f"\nError de PortAudio: {pae}", file=sys.stderr); print("Verifica el micrófono.", file=sys.stderr)
         try: print(sd.query_devices())
         except Exception: pass; return None
    except Exception as e: print(f"\nError en stream de audio: {e}", file=sys.stderr); return None

def save_audio_to_wav_bytes(audio_data: bytes, sample_rate: int) -> bytes:
    """Guarda los datos de audio (bytes, int16) en formato WAV en memoria."""
    bytes_io = io.BytesIO()
    with wave.open(bytes_io, 'wb') as wf:
        wf.setnchannels(1); wf.setsampwidth(2); wf.setframerate(sample_rate)
        wf.writeframes(audio_data)
    return bytes_io.getvalue()

async def transcribe_audio_bytes(audio_bytes: bytes, model, sample_rate: int):
    """Transcribe datos de audio (WAV en bytes) usando Whisper, con reducción de ruido opcional."""
    if not audio_bytes: return None
    print("Procesando audio (reducción de ruido y transcripción)...", flush=True)
    try:
        # Cargar audio WAV desde bytes
        with io.BytesIO(audio_bytes) as wav_buffer:
            samplerate_wav, data_wav = wav.read(wav_buffer)

        # Convertir a float32 para noisereduce y whisper
        audio_np = data_wav.astype(np.float32) / 32768.0

        # --- Aplicar Reducción de Ruido ---
        # Nota: 'prop_decrease' controla cuánto reducir el ruido, ajustar si es necesario
        # 'sr' debe coincidir con la tasa de muestreo del audio
        print("Aplicando reducción de ruido...", flush=True)
        reduced_noise_audio = await asyncio.to_thread(
            nr.reduce_noise,
            y=audio_np,
            sr=sample_rate,
            prop_decrease=0.8 # Reduce el ruido estimado en un 80% (ajustable)
        )
        print("Reducción de ruido completada.", flush=True)
        # ----------------------------------

        # Transcribir el audio LIMPIO
        print("Transcribiendo audio limpio...", flush=True)
        result = await asyncio.to_thread(
            model.transcribe,
            reduced_noise_audio, # <--- Usar el audio procesado
            language='es',
            fp16=False
        )
        transcription = result["text"].strip()
        if transcription: print(f"Texto reconocido: '{transcription}'", flush=True)
        return transcription
    except Exception as e:
        print(f"Error durante reducción de ruido o transcripción: {e}", flush=True)
        return None

# --- MODIFICACIÓN CLAVE: Función LLM sin historial ---
async def get_llm_response_stateless(prompt, model_name):
    """Obtiene una respuesta del modelo Ollama tratando cada prompt como independiente."""
    global SYSTEM_PROMPT # Acceder al prompt del sistema global
    print(f"Pensando (modelo: {model_name})...", flush=True)
    try:
        # Crear la lista de mensajes SOLO con el prompt del sistema y el prompt actual
        messages = [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': prompt}
        ]

        # Llamar a ollama.chat con esta lista limitada de mensajes
        response = await asyncio.to_thread(
             ollama.chat,
             model=model_name,
             messages=messages # <- Solo system + user actual
        )

        llm_response = response['message']['content']
        # Ya NO añadimos nada al historial aquí
        print(f"Respuesta de Ollama: '{llm_response}'", flush=True)
        return llm_response
    except Exception as e:
        print(f"Error contactando con Ollama: {e}", flush=True)
        # Ya no hay historial que limpiar
        return "Lo siento, hubo un error al contactar el modelo de lenguaje."

async def speak_text_edge(text, voice, output_filename):
    """Genera y reproduce audio TTS usando edge-tts y pydub+sounddevice."""
    if not text: print("TTS: Texto vacío, no hay nada que decir."); return
    print("Generando voz...", flush=True)
    temp_tts_file = output_filename
    try:
        communicate = edge_tts.Communicate(text, voice, rate="+10%")
        await communicate.save(temp_tts_file)
        print("Hablando...", flush=True)
        def play_audio():
            try:
                audio = AudioSegment.from_mp3(temp_tts_file)
                samples = np.array(audio.get_array_of_samples())
                if audio.channels > 1: samples = samples.reshape((-1, audio.channels))
                sd.play(samples, audio.frame_rate, blocking=True)
            except FileNotFoundError: print(f"Error Pydub: No se pudo cargar {temp_tts_file}. ¿ffmpeg instalado?", file=sys.stderr)
            except NameError: print("Error: 'AudioSegment' no está definido. ¿Falta import?", file=sys.stderr)
            except Exception as play_err: print(f"Error al reproducir audio: {play_err}", file=sys.stderr)
        await asyncio.to_thread(play_audio)
    except edge_tts.exceptions.NoAudioReceived: print("Error TTS: No se recibió audio. Verifica conexión/voz.", file=sys.stderr)
    except Exception as e: print(f"Error en síntesis de voz: {e}", file=sys.stderr)
    finally:
        if os.path.exists(temp_tts_file):
            try: os.remove(temp_tts_file)
            except Exception: pass

In [13]:
# --- Bucle Principal Asíncrono (Modificado para no usar historial) ---
async def main_loop():
    # Ya NO inicializamos conversation_history aquí

    print("\n--- Asistente NO Conversacional Iniciado (Notebook) ---")
    print("Cada pregunta se trata de forma independiente.")
    print(f"Modelo LLM: {OLLAMA_MODEL}")
    print(f"Modelo Whisper: {WHISPER_MODEL}")
    print(f"Voz TTS: {TTS_VOICE}")
    print(f"Di '{', '.join(EXIT_KEYWORDS)}' para salir.")
    print("-------------------------------------------")

    try: await speak_text_edge("Hola, ¿en qué puedo ayudarte?", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
    except Exception as initial_speak_error: print(f"Error en saludo inicial: {initial_speak_error}")

    while True:
        try:
            audio_data = await listen_for_speech()
            if audio_data is None: print("Reintentando escucha..."); await asyncio.sleep(0.5); continue

            wav_bytes = save_audio_to_wav_bytes(audio_data, VAD_SAMPLE_RATE)
            user_text = await transcribe_audio_bytes(wav_bytes, whisper_model, VAD_SAMPLE_RATE)

            if not user_text: print("No se entendió el audio."); continue

            # --- Lógica de Salida (sin cambios) ---
            cleaned_text_for_exit = user_text.lower().strip()
            translator = str.maketrans('', '', string.punctuation)
            cleaned_text_no_punct = cleaned_text_for_exit.translate(translator).strip()
            words_in_text = cleaned_text_no_punct.split()
            is_exit = any(keyword in words_in_text for keyword in EXIT_KEYWORDS)

            if is_exit:
                 print("Detectada palabra clave de salida.")
                 await speak_text_edge("Entendido. ¡Hasta pronto!", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
                 break # Salir del bucle while

            # --- MODIFICACIÓN: Llamar a la función LLM sin historial ---
            ai_response = await get_llm_response_stateless(user_text, OLLAMA_MODEL)
            # --------------------------------------------------------

            await speak_text_edge(ai_response, TTS_VOICE, TEMP_AUDIO_FILE_TTS)

        except asyncio.CancelledError: print("Bucle principal cancelado."); break
        except KeyboardInterrupt:
             print("\nInterrupción detectada en el bucle. Saliendo...")
             try: await speak_text_edge("Detectada interrupción. ¡Adiós!", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
             except: pass
             break
        except Exception as loop_error:
             print(f"\nError inesperado en el bucle: {loop_error}")
             try: await speak_text_edge("Lo siento, ocurrió un error inesperado.", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
             except: pass
             await asyncio.sleep(1)

In [14]:
# En esta nueva celda, ejecuta esto para iniciar:
try:
    await main_loop()
except (KeyboardInterrupt, asyncio.CancelledError):
    print("\nEjecución interrumpida por el usuario.")
finally:
    # Limpieza final
    if os.path.exists(TEMP_AUDIO_FILE_TTS):
        try: os.remove(TEMP_AUDIO_FILE_TTS)
        except Exception: pass
    print("--- Asistente Conversacional (Notebook) Terminado ---")


--- Asistente NO Conversacional Iniciado (Notebook) ---
Cada pregunta se trata de forma independiente.
Modelo LLM: gemma3
Modelo Whisper: small
Voz TTS: es-ES-ElviraNeural
Di 'adiós, terminar, salir, hasta luego, bye, exit, adios' para salir.
-------------------------------------------
Generando voz...
Hablando...
🎤 Escuchando...
Bucle principal cancelado.
--- Asistente Conversacional (Notebook) Terminado ---
